In [51]:
import time
import json

import networkx as nx
import twitter
import matplotlib.pyplot as plt

In [7]:
# read keys and tokens for Twitter API

with open('consumer_key.key') as f:
    consumer_key = f.read().strip()
    
with open('consumer_secret.key') as f:
    consumer_secret = f.read().strip()
    
with open('access_token.token') as f:
    access_token = f.read().strip()
    
with open('access_token_secret.token') as f:
    access_token_secret = f.read().strip()
    

In [10]:
api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_token,
                  access_token_secret=access_token_secret)

In [23]:
try:
    cache_dict = json.load(open('graph_cache.json', 'r'))
except FileNotFoundError:
    cache_dict = {}

In [65]:
def construct_graph(user_id, output_graph, api, waves=3):
    output_graph.add_node(user_id)
    if waves != 0:
        global cache_dict
        
        if user_id in cache_dict:
            friends = cache_dict[user_id]
        else:
            try:
                friends = api.GetFriendIDs(user_id=user_id, total_count=1000)
            except twitter.TwitterError as err:
                if err.message[0]['message'] == 'Rate limit exceeded':
                    print('Sleeping')
                    time.sleep(15 * 60)
                friends = api.GetFriendIDs(user_id=user_id, total_count=1000)
            cache_dict[user_id] = friends
            
        for friend in friends:
            if friend not in output_graph.nodes():
                construct_graph(friend, output_graph, api, waves - 1)
            output_graph.add_edge(user_id, friend)

In [ ]:
# start with my (@lapsya) id of 192217140 for now
main_graph = nx.DiGraph()
construct_graph(192217140, main_graph, api)

Sleeping


In [ ]:
main_graph.nodes()

In [ ]:
cache_dict

In [ ]:
with open('graph_cache.json', 'w') as f:
    json.dump(cache_dict, f)

In [ ]:
nx.draw(main_graph)
plt.show()